In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

  Using cached ml_dtypes-0.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.3.2 which is incompatible.
2.16.0-dev20240202


In [2]:
!pip install tensorflow-text
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

  Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.16.0.dev20240202 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.


In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-02-03 13:40:37--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2024-02-03 13:40:37 (9.46 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2024-02-03 13:40:37--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

In [4]:
df_train = pd.read_csv(train_file_path, sep="\t", header=None, names=["spam", "message"], encoding="utf-8")
df_test = pd.read_csv(test_file_path, sep="\t", header=None, names=["spam", "message"])

In [5]:
train_labels = df_train.pop("spam")
test_labels = df_test.pop("spam")

In [6]:
train_labels = pd.Categorical(train_labels, categories=["ham", "spam"]).codes.astype("float32")
test_labels = pd.Categorical(test_labels, categories=["ham", "spam"]).codes.astype("float32")

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.message)

In [8]:
train_seq = tokenizer.texts_to_sequences(df_train.message)
train_seq = pad_sequences(train_seq, padding="pre", truncating="post")
train_seq.shape

(4179, 189)

In [9]:
seq_len = train_seq.shape[1]

In [10]:
test_seq = tokenizer.texts_to_sequences(df_test.message)
test_seq = pad_sequences(test_seq, maxlen=seq_len, padding="pre", truncating="post")
test_seq.shape

(1392, 189)

In [11]:
model = keras.Sequential([
    keras.layers.Embedding(len(tokenizer.word_index), 100),
    keras.layers.LSTM(units=128,
                      recurrent_initializer="glorot_uniform"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [12]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [13]:
train_labels.shape

(4179,)

In [14]:
model.fit(train_seq, train_labels,
          batch_size=32,
          epochs=10,
          validation_data=(test_seq, test_labels))

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.9038 - loss: 0.2830 - val_accuracy: 0.9837 - val_loss: 0.0606
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9941 - loss: 0.0194 - val_accuracy: 0.9886 - val_loss: 0.0467
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9987 - loss: 0.0075 - val_accuracy: 0.9886 - val_loss: 0.0407
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9999 - loss: 0.0019 - val_accuracy: 0.9879 - val_loss: 0.0548
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9998 - loss: 8.2423e-04 - val_accuracy: 0.9858 - val_loss: 0.0688
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 1.0000 - loss: 1.9772e-04 - val_accuracy: 0.9886 - val_loss: 0.0656
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 1.0000 - loss: 7.7387e-05 - val_accuracy: 0.9879 - val_loss: 0.0743
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 1.0000 - loss: 3.49

In [15]:
test_msg = ["how are you doing today",
            "sale today! to stop texts call 98912460324",
            "i dont want to go. can we try it a different day? available sat",
            "our new mobile video service is live. just install on your phone to start watching.",
            "you have won £1000 cash! call to claim your prize.",
            "i'll bring it tomorrow. don't forget the milk.",
            "wow, is your arm alright. that happened to me one time too"
          ]
test_msg = ["how are you doing today?"]

pred_seq = tokenizer.texts_to_sequences(test_msg)
print(pred_seq)
pred_seq = pad_sequences(pred_seq, maxlen=seq_len, padding="pre", truncating="post")
# pred_seq.shape

[[49, 22, 3, 158, 90]]


In [16]:
pred_prob = model.predict(pred_seq)
pred_label = ["spam" if p > 0.1 else "ham" for p in pred_prob]
prediction = [[p.item(), l] for p, l in zip(pred_prob, pred_label)]
print(prediction[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
[2.85676065914231e-07, 'ham']


In [17]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_seq = tokenizer.texts_to_sequences([pred_text])
  pred_seq = pad_sequences(pred_seq, maxlen=seq_len, padding="pre", truncating="post")
  pred_prob = model.predict(pred_seq)
  pred_label = ["spam" if p > 0.1 else "ham" for p in pred_prob]
  prediction = [[p.item(), l] for p, l in zip(pred_prob, pred_label)]

  return prediction[0]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[2.85676065914231e-07, 'ham']


In [18]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
You passed the challenge. Great job!
